In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id = '1NGr179sHIt30XmQOEdJItlFRwgcnNFRh'
file = drive.CreateFile({'id':id}) 
file.GetContentFile('all_review.csv')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load in Dataset

In [ ]:
df=pd.read_csv('all_review.csv', index_col=False)
df.head()

,Name,Review,Rating,#ThumbsUp,Date&Time,App,language
0,linah sibanda,on this app i cant find a partner,5,0,18-02-2022 01:19,Tinder,en
1,Norman Johnson,tinder would be so much good if we could speci...,3,0,18-02-2022 01:16,Tinder,en
2,David Hume,still do not correctly notify match or message...,1,0,18-02-2022 01:11,Tinder,en
3,Last 1 Standing,get block because i update my bio to say `` i ...,2,0,18-02-2022 01:11,Tinder,en
4,Arthur Magamedov,love it !,5,0,18-02-2022 01:06,Tinder,en


In [ ]:
#focus on reviews that have at least one thumbup
df=df[df['#ThumbsUp']>=1].reset_index()

In [ ]:
df.shape

(110966, 8)

In [ ]:
df.App.unique()

array(['Tinder', 'Bumble', 'Hinge'], dtype=object)

In [ ]:
bumble_nega_df=df[((df.Rating==1)|(df.Rating==2)|(df.Rating==3))&(df.App=='Bumble')]
bumble_nega_reviews = bumble_nega_df['Review'].tolist()

In [ ]:
bumble_nega_df.shape

(26034, 8)

In [ ]:
hinge_nega_df=df[((df.Rating==1)|(df.Rating==2)|(df.Rating==3))&(df.App=='Hinge')]
hinge_nega_reviews = hinge_nega_df['Review'].tolist()

In [ ]:
hinge_nega_df.shape

(8504, 8)

In [ ]:
tinder_nega_df=df[((df.Rating==1)|(df.Rating==2)|(df.Rating==3))&(df.App=='Tinder')]
tinder_nega_reviews = tinder_nega_df['Review'].tolist()

In [ ]:
tinder_nega_df.shape

(59381, 8)

# Install packages and load in model

In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 11.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.7 MB/s 
     |████████████████████████████████| 77 kB 3.9 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
     |████████████████████████████████| 880 kB 46.2 MB/s 
     |████████████████████████████████| 6.6 MB 31.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=875aa18d4220065e7e9992cea863b7653ae54aa4c897f5d6bf06aa43a81ad397
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=f91e8e49d0c28a1658d256646541f225d5401db5fe6b5bfb86a3346d70583995
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sentence-transformers sa

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

# Embeddings

In [ ]:
embeddings = model.encode(tinder_nega_reviews, show_progress_bar=True)

Batches:   0%|          | 0/1856 [00:00<?, ?it/s]

In [ ]:
embeddings.shape

(59381, 768)

# Dimensionality reduction

In [ ]:
!pip install umap-learn

     |████████████████████████████████| 88 kB 3.2 MB/s 
     |████████████████████████████████| 1.1 MB 43.9 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=8d8822f5772d9ca70e18435fc7ba7a93558d512945f741d1084719618d45852a
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.6-py3-none-any.whl size=53943 sha256=b43ddeaa4e43089fdb78d689a0ca012e3943217d0065ddc3b98766b94b4d2a32
  Stored in directory: /root/.cache/pip/wheels/03/f1/56/f80d72741e400345b5a5b50ec3d929aca581bf45e0225d5c50
Successfully built umap-learn pynndescent


In [ ]:
import umap
umap_embeddings = umap.UMAP(n_neighbors=15, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

# Clustering

In [ ]:
!pip install hdbscan

     |████████████████████████████████| 5.2 MB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330776 sha256=c5dd453c751da78d59e0a62a2bd5632c9030c0c0d748f25f1f32638c09471289
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef4a71e896b62f5161f2bc8a14c97a
Successfully built hdbscan


In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=15,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

# C-TF-IDF

In [ ]:
docs_df = pd.DataFrame(hinge_nega_reviews, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(2, 2)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(bumble_nega_reviews))

# Topic representation

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

## Bumble Negative Reviews - BERT

In [ ]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
3,2,22595
0,-1,3399
2,1,24
1,0,16


In [ ]:
top_n_words[2][:10]

[('waste time', 0.009819360872065296),
 ('use app', 0.009232755384866101),
 ('dating app', 0.008085215773722409),
 ('swipe right', 0.0071895500914358436),
 ('24 hour', 0.006394564104604394),
 ('scam profile', 0.005514527564866923),
 ('customer service', 0.005334709781694326),
 ('dating apps', 0.005127346110392925),
 ('good app', 0.004945825068977172),
 ('cancel subscription', 0.0045258950187969105)]

In [ ]:
top_n_words[1][:10]

[('robot robot', 0.7574051272770258),
 ('robot scam', 0.2953142883666955),
 ('robot app', 0.2141467407764752),
 ('love robot', 0.2141467407764752),
 ('spam robot', 0.20829774458553874),
 ('profile robot', 0.2006376399240277),
 ('90 robot', 0.14319941767642824),
 ('iceberg selfies', 0.14319941767642824),
 ('robot flake', 0.14319941767642824),
 ('account run', 0.14319941767642824)]

In [ ]:
top_n_words[0][:10]

[('just look', 0.06882141807787608),
 ('love people', 0.06275872449869356),
 ('save profile', 0.057767946903559124),
 ('set quick', 0.057767946903559124),
 ('adults wo', 0.057767946903559124),
 ('costly just', 0.057767946903559124),
 ('extremely undesirable', 0.057767946903559124),
 ('physical attribute', 0.057767946903559124),
 ('penalty right', 0.057767946903559124),
 ('spam ok', 0.057767946903559124)]

## Tinder Negative Reviews - BERT

In [ ]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
1,1,59302
0,0,79


In [ ]:
top_n_words[1][:10]

[('use app', 0.006670208821270627),
 ('waste time', 0.006039938246599703),
 ('scam profile', 0.005871793182413687),
 ('block reason', 0.005299213905764053),
 ('account block', 0.005080362723672028),
 ('phone number', 0.00506428731352051),
 ('tinder gold', 0.004782527668692125),
 ('super love', 0.004679727417991262),
 ('wo let', 0.004352088410994776),
 ('customer service', 0.004182761941008215)]

In [ ]:
top_n_words[0][:10]

[('south korea', 0.11100853599526314),
 ('term condition', 0.10141503554842374),
 ('outside south', 0.05474181133665216),
 ('mandatory term', 0.04206097732064675),
 ('agree term', 0.04158991804846778),
 ('new term', 0.03895410762022283),
 ('korean language', 0.038864961419017865),
 ('personal information', 0.03830895286019261),
 ('personal data', 0.034434240647171384),
 ('share personal', 0.03351610058403696)]

## Hinge Negative Reviews - BERT

In [ ]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
0,-1,6186
23,22,494
6,5,221
16,15,199
31,30,185
22,21,124
15,14,113
30,29,108
11,10,83
26,25,82


In [ ]:
top_n_words[22][:10]

[('dating apps', 0.024310961019000845),
 ('dating app', 0.01967321644374677),
 ('love app', 0.0164533973722181),
 ('good app', 0.016064006761572755),
 ('app good', 0.01536665267503097),
 ('good dating', 0.009657285289315082),
 ('love dating', 0.009554055631465518),
 ('people love', 0.009310180411589937),
 ('feel love', 0.009296194496756333),
 ('app love', 0.009135483640990847)]

In [ ]:
top_n_words[5][:10]

[('fat female', 0.05035276286674588),
 ('obese female', 0.04457087096051056),
 ('fat chick', 0.04390403868329275),
 ('morbidly obese', 0.033072434684787586),
 ('fat people', 0.029006404397551463),
 ('overweight female', 0.02654347567856401),
 ('female app', 0.02291224297137234),
 ('body type', 0.02238229023859556),
 ('female fat', 0.021456597875923324),
 ('love fat', 0.018592314405571127)]

In [ ]:
top_n_words[15][:10]

[('use app', 0.03050913837290455),
 ('customer service', 0.025472504905923628),
 ('phone number', 0.020233122382015387),
 ('contact support', 0.015740325987688264),
 ('age restriction', 0.014931839892890125),
 ('driver license', 0.014521889775673102),
 ('send id', 0.012985874803150967),
 ('send email', 0.012472917851575968),
 ('delete account', 0.011690051940503005),
 ('try contact', 0.011613653250025655)]

In [ ]:
top_n_words[30][:10]

[('block reason', 0.10667700782830304),
 ('got block', 0.0597352517573054),
 ('wo load', 0.05691021259952389),
 ('wo let', 0.04190708198049846),
 ('reason explanation', 0.041441383318700384),
 ('wo explain', 0.0398325609064054),
 ('download block', 0.0385689500464415),
 ('block violate', 0.03855718721271037),
 ('absolutely reason', 0.03247980936802324),
 ('block absolutely', 0.032213522595769505)]

In [ ]:
top_n_words[21][:10]

[('love day', 0.08024662214752845),
 ('10 love', 0.06091056869272371),
 ('rise week', 0.025498691066475093),
 ('love 10', 0.024888494362985337),
 ('design delete', 0.023682920150126133),
 ('love people', 0.020350112632626347),
 ('people day', 0.01755695297426815),
 ('free love', 0.017293370526544505),
 ('pay love', 0.016056134457752447),
 ('day unless', 0.01498122518748175)]

In [ ]:
top_n_words[14][:10]

[('pay month', 0.026387999846578976),
 ('month subscription', 0.025929121690753044),
 ('customer service', 0.01736139871529606),
 ('use app', 0.016774018218915943),
 ('app year', 0.01402700858281422),
 ('support team', 0.013971207323086486),
 ('month later', 0.012438744772929118),
 ('email support', 0.012235864920965274),
 ('month pay', 0.01177407591259734),
 ('app month', 0.01151000543634629)]

In [ ]:
top_n_words[29][:10]

[('just bad', 0.08273361422502072),
 ('bad app', 0.07096688947197294),
 ('notification suck', 0.058481552311521356),
 ('algorithm lot', 0.058481552311521356),
 ('bad terrible', 0.05597868127381664),
 ('horrible algorithm', 0.0542028660117686),
 ('terrible algorithm', 0.05169999497406389),
 ('horrible experience', 0.05169999497406389),
 ('constantly crash', 0.04919712393635917),
 ('app awful', 0.04795841593967923)]

In [ ]:
top_n_words[10][:10]

[('mile away', 0.11361228365734294),
 ('100 mile', 0.055159710179674575),
 ('year old', 0.044406835670626435),
 ('50 mile', 0.0386445096226819),
 ('30 mile', 0.02878864823665538),
 ('age range', 0.023981522963602637),
 ('away set', 0.022490870726770217),
 ('set age', 0.020085427352224596),
 ('deal breaker', 0.019263769189167855),
 ('match people', 0.018912176812111704)]

In [ ]:
top_n_words[25][:10]

[('use facebook', 0.11491433589556399),
 ('phone number', 0.1117181841944232),
 ('facebook account', 0.08996820673023996),
 ('sign facebook', 0.05387593876080438),
 ('use phone', 0.041496021332486484),
 ('facebook login', 0.03662410042843304),
 ('account use', 0.03600955144502548),
 ('use app', 0.034853320105216655),
 ('facebook phone', 0.03431639030120357),
 ('log facebook', 0.03411595497622687)]

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
np.savetxt("tinder_nega_embeddings.csv", embeddings, delimiter=",")
!cp tinder_nega_embeddings.csv "drive/My Drive/"

In [ ]:
id = '1-1-5cC_Az4eQIZ-D4zrob0m81X1Tof76'
file = drive.CreateFile({'id':id}) 
file.GetContentFile('hinge_nega_embeddings.csv')

In [ ]:
test=pd.read_csv('hinge_nega_embeddings.csv', header=None)
embeddings=test.to_numpy()

In [ ]:
embeddings.shape

(8504, 768)

## Reference:
[Topic Modeling with BERT](https://towardsdatascience.com/topic-modeling-with-bert-779f7db187e6)